# TensorFlow Tutorial #20
# 自然语言处理

by [Magnus Erik Hvass Pedersen](http://www.hvass-labs.org/)/[GitHub中文](https://github.com/Hvass-Labs/TensorFlow-Tutorials-Chinese)
/ [GitHub](https://github.com/Hvass-Labs/TensorFlow-Tutorials) / [Videos on YouTube](https://www.youtube.com/playlist?list=PL9Hr9sNUjfsmEu1ZniY0XpHSzl5uihcXZ)

中文翻译[ZhouGeorge](https://github.com/ZhouGeorge)

## 介绍

本教程是关于自然语言处理（NLP）的基础应用-情感分析，在这个过程中，我们会尝试把电影评论分类为正面或负面的。

一个简单的例子："This movie is not very good."，文本的结尾是"very good"，这表明了一种非常积极的情感，但是它被否定了，因为它前面有 "not"这个词，所以这个文本应该被分类成负面的情感。我们怎样才能教一个神经网络来做这个分类呢？

另一个问题是不能直接用于文本数据，所以我们需要将文本转换成能被神经网络接受的数字。

还有另一个问题是文本的长度是任意的。我们之前教程中使用的神经网络的输入是固定的形状-除了第一维是用于改变batch-size。现在我们需要一种神经网络可以输入不同长度的文本。

一般来说，你应该先熟悉TensorFlow和Keras，见教程 #01 和 #03-C

## 流程图

为了解决这个问题，我们需要几个步骤。首先，我们需要将原始文本数据转换成整数形代号（integer-tokens）。这些代号实际上只是整个词汇表索引的列表。为了将相似含义的词映射到相似的向量，我们将这些整形的代号转换到称为嵌入的实值向量（embedding-vectors），它的映射将用于神经网络的训练。之后，我们将这些嵌入向量输入（embedding-vectors）到循环神经网络，循环神经网络可以将任意长度的序列作为输入并输出关于输入内容的总结。我们将这个结果传入Sigmoid函数后会得到一个在0.0到1.0之间的值，0.0被认为是负面的情感，1.0代表正面的情感。这整个过程可以让我们将输入文本分类成负面或正面的情感。

算法的流程图大致如下：

<img src="images/20_natural_language_flowchart.png" alt="Flowchart NLP" style="width: 300px;"/>

## 循环神经网络

循环神经网络 (RNN)的基础构筑模块是循环单元(RU)。循环单元的变体有很多，比如相笨重的LSTM（Long-Short-Term-Memory）和我们将在个份教程中使用的稍微简单一点的GRU（Gated Recurrent Unit）。文献中的实验表明，LSTM和GRU的性能大致相同。甚至存在更简单的变体，它们的表现可能比LSTM和GRU更好，但是我们这个教程要用Keras，其他的变体没有在Keras中被实现。

下面的图展示了循环单元的抽象概念，它有一个内部状态（internal state），每次当单元接收到一个新的输入时状态都会被更新。内部状态被视为一种记忆。然而，它不同于传统的计算机内存仅对每个位进行开或关的方式操作。相反的，循环单元将浮点值存储在其内存状态中，使用矩阵运算来读取和写入，因此操作都是可微的。这意味着记忆状态可以存储任意浮点值（尽管通常在-1.0和1.0之间）并且这个网络也可以像正常的神经网络一样用梯度下降法训练。

新的状态值取决于过去的状态值和当前的输入。比如，如果一个状态已经被记住了，我们最近看到的词是 "not"并且当前的输入是"good" ，然后我们需要存储新的状态，记住 "not good"表示负面情感。

循环单元用于将过去的状态和新的输入映射到新的状态部分，被称为gate，但是它实际只是一种矩阵运算。这里还有另一个gate用于计算循环单元的输出值。这些gata的实现方式因不同类型的循环单元而异。这幅图片仅仅展示了大概。LSTM比GRU有更多的gate，但有一些显得多余，可以被忽略。

为了训练这个循环单元，我们必须逐渐改变gate的权重矩阵，那么在一个输入序列后循环单元会给我们想要的输出。这些都会在TensorFlow中自动地完成。

![Recurrent unit](images/20_recurrent_unit.png)

### 展开的网络

另一种形象化和理解一个循环神经网络的方法是展开它的递归。这幅图只是一个循环单元（RU）的展示，它将在一系列的时间步长中从输入序列中接收文本字。

每一次新的序列开始，RU的初始记忆状态会被Keras/TensorFlow内部重置为0。

第一个时间步长中，"this"这个词被输入到RU，然后用RU的内部状态（初始是0）和它的gate计算出新的状态。这个RU也用它另一个gate去计算输出，但是输出在这里被无视了，因为只有在序列的结尾才需要输出一个总结。

在第二个时间步长中 "is"这个词被输入到RU，而现在使用的内部状态是根据上一个词"this"刚刚更新过的。

"this is"并没有什么意义，所以RU见过这些词后可能不会在它的内部状态中保存任何重要的东西。但是当它看到第三个词 "not"，RU已经了解到它可能对确定输入文本的整体情感很重要，所以它需要被存到RU的记忆状态中，当RU看见第六个时间步长的词"good"时，会被用到。

最后整个序列被处理完后，RU输出一个向量的值，是关于它所看到的输入序列的总结。然后我们用带有Sigmoid激活函数的全连接层去获得一个0.0到1.0间独立的值，我们用他来解读近0.0）是正面的（值接近1.0）。

注意，为了清晰起见，这个图并没有显示从文本（text-words）到整数代号（integer-tokens）和嵌入向量（embedding-vectors）的映射，也没有显示最后输出前的的全连接层和Sigmoid激活函数。

![Unrolled network](images/20_unrolled_flowchart.png)

### 3层展开的网络

在这份教程中我们使用具有3个循环单元（层）RU1,RU2,RU3的循环神经网络，“展开”图如下。
第一层有点像上图的单层RNN。第一个循环单元RU1内部状态被Keras / TensorFlow初始化为0。然后"this"被输入到RU1并且更新它的内部状态。然后处理下个词"is"，等等。但是不同于在序列最后输出一个单独的总结，我们要用到RU1每一个时间步长的输出。这就生成了一个新的序列，它可作为下一个循环单元RU2的输入。第二层重复同样的过程并且也产生一个新的序列，可以作为第三个循环单元RU3的输入，而RU3的输出被传到具有Sigmoid的全连接层，输出的结果是0.0（负面情感）到1.0（正面情感）。

注意，为了清晰起见，从文本（text-words）到整数代号（integer-tokens）和嵌入向量（embedding-vectors）的映射在下面的图中被省略了。


![Unrolled 3-layer network](images/20_unrolled_3layers_flowchart.png)

### 梯度消失和梯度爆炸

为了训练循环单元里gate的权重，我们必须最小一些化损失函数，它们测量网络的实际输出与期望输出之间的差异。

从上面的展开图我们可以看到，在输入序列中每个词都递归用到循环单元，这意味着每一个时间步长都要重复一次gate的计算。梯度信号必须从损失函数中回流到第一次计算的gate，如果循环gate的梯度计算是乘法的，那么这个传递的过程是指数级别的函数。

在这个教程中我们将使用超过500词的文本。这意味着RU的gate更新它的内部记忆会递归超过500次。如果梯度是1.01，自乘500次，结果大约是145。如果梯度是0.99，自乘500次，结果大约是0.007 。这就称为梯度爆炸和梯度消失。唯一能在重复乘法中幸存的梯度是0和1。

为了避免梯度消失和梯度爆炸，在设计循环单元和它的gate时必须要小心。这就是为什么GRU的实际实现更加复杂，因为它试图通过没有这种扭曲的方式将梯度传送回gate。

## 引用

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from scipy.spatial.distance import cdist

/home/magnus/anaconda3/envs/tf-gpu/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


我们需要从Keras中引入一些模块。

In [2]:
# from tf.keras.models import Sequential  # This does not work!
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU, Embedding
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

开发环境Python 3.6（Anaconda）和其他安装包的版本：

In [3]:
tf.__version__

'1.5.0'

In [4]:
tf.keras.__version__

'2.1.2-tf'

## 加载数据
我们将使用从IMDB获取50000条影评来作为数据集。Keras内置一个函数可以下载相似的数据集（但是只有一半的大小）。然而，Keras的版本已经将数据集中的文本转换成整数代号，这是处理自然语言的一个关键部分，本教程中将做一个演示，所以我们下载了原始的文本数据。

注意：数据集有84MB，它会被自动的下载。


In [5]:
import imdb

如果你想让文件保存到其他地址，你可以改变它。

In [6]:
# imdb.data_dir = "data/IMDB/"

自动下载和解压文件。

In [7]:
imdb.maybe_download_and_extract()

Data has apparently already been downloaded and unpacked.


加载训练集和测试集。

In [8]:
x_train_text, y_train = imdb.load_data(train=True)
x_test_text, y_test = imdb.load_data(train=False)

In [9]:
print("Train-set size: ", len(x_train_text))
print("Test-set size:  ", len(x_test_text))

Train-set size:  25000
Test-set size:   25000


为了进行下面的处理，将它们合并成一个数据集。

In [10]:
data_text = x_train_text + x_test_text

输出训练集的一个例子，检测数据是否正确。

In [11]:
x_train_text[1]

'A simple comment...<br /><br />What can I say... this is a wonderful film that I can watch over and over. It is definitely one of the top ten comedies made. With a great cast, Jack Lemmon and Walter Matthau wording a perfect script by Neil Simon, based on his play.<br /><br />It is real to life situation done perfectly. If you have digital cable, one gets the menu on bottom of screen to give what is on. It usually gives this film ***% stars but in reality it deserves **** stars. If you really watch this film, one can tell that it will be as funny and fresh a hundred years from now.'

正确的“类别”是影评的情感。0.0是负面的情感，1.0是正面的情感。在上面例子中，评论是积极的。

In [12]:
y_train[1]

1.0

## 分词器（tokenizer）

神经网络不能直接处理字符串文本，所以我们必须转换它。这个过程有两步，第一步称为“分词器”，将数据集输入神经网络之前将单词转换成整形。第二步是属于神经网络自身处理的一部分，它被称为“embedding”层，它会在之后被介绍。

我们可以指定分词器只使用数据集中10000个最常用的词

In [13]:
num_words = 10000

In [14]:
tokenizer = Tokenizer(num_words=num_words)

分词器可以“拟合”到数据集。它扫描所有的文本，将不需要的字符剥离出来（比如标点符号）并转换转为小写字符。然后分词器构建一个所有独特的词的词汇表，用于访问数据。

注意，我们分词器“拟合”全部的数据集，所以它包括了训练集和测试集。这样做没问题是因为我们只是建立一个尽可能完整的词汇表。实际的神经网络当然只能在训练集上训练。

In [15]:
%%time
tokenizer.fit_on_texts(data_text)

CPU times: user 10.6 s, sys: 16 ms, total: 10.6 s
Wall time: 10.6 s


如果你喜欢使用完整的词汇表，设置上面的 `num_words=None`，然后它将自动设置词汇表大小。

In [16]:
if num_words is None:
    num_words = len(tokenizer.word_index)

然后我们可以检查由分词器收集的词汇表。这是由数据集中的单词出现次数排序的。这些整形数字被称为词索引或者是代号（tokens），因为它们在词汇表中唯一地识别每个单词。

In [17]:
tokenizer.word_index

{'the': 1,
 'and': 2,
 'a': 3,
 'of': 4,
 'to': 5,
 'is': 6,
 'br': 7,
 'in': 8,
 'it': 9,
 'i': 10,
 'this': 11,
 'that': 12,
 'was': 13,
 'as': 14,
 'for': 15,
 'with': 16,
 'movie': 17,
 'but': 18,
 'film': 19,
 'on': 20,
 'not': 21,
 'you': 22,
 'are': 23,
 'his': 24,
 'have': 25,
 'be': 26,
 'one': 27,
 'he': 28,
 'all': 29,
 'at': 30,
 'by': 31,
 'an': 32,
 'they': 33,
 'so': 34,
 'who': 35,
 'from': 36,
 'like': 37,
 'or': 38,
 'just': 39,
 'her': 40,
 'out': 41,
 'about': 42,
 'if': 43,
 "it's": 44,
 'has': 45,
 'there': 46,
 'some': 47,
 'what': 48,
 'good': 49,
 'when': 50,
 'more': 51,
 'very': 52,
 'up': 53,
 'no': 54,
 'time': 55,
 'my': 56,
 'even': 57,
 'would': 58,
 'she': 59,
 'which': 60,
 'only': 61,
 'really': 62,
 'see': 63,
 'story': 64,
 'their': 65,
 'had': 66,
 'can': 67,
 'me': 68,
 'well': 69,
 'were': 70,
 'than': 71,
 'much': 72,
 'we': 73,
 'bad': 74,
 'been': 75,
 'get': 76,
 'do': 77,
 'great': 78,
 'other': 79,
 'will': 80,
 'also': 81,
 'into': 82,
 'p

然后我们用分词器去将训练集的文本转换成代号形式的列表。

In [18]:
x_train_tokens = tokenizer.texts_to_sequences(x_train_text)

举个例子，下面是训练级中的一个文本：

In [19]:
x_train_text[1]

'A simple comment...<br /><br />What can I say... this is a wonderful film that I can watch over and over. It is definitely one of the top ten comedies made. With a great cast, Jack Lemmon and Walter Matthau wording a perfect script by Neil Simon, based on his play.<br /><br />It is real to life situation done perfectly. If you have digital cable, one gets the menu on bottom of screen to give what is on. It usually gives this film ***% stars but in reality it deserves **** stars. If you really watch this film, one can tell that it will be as funny and fresh a hundred years from now.'

此文本对应于下列代号形式的列表：

In [20]:
np.array(x_train_tokens[1])

array([   3,  591,  929,    7,    7,   48,   67,   10,  131,   11,    6,
          3,  393,   19,   12,   10,   67,  103,  121,    2,  121,    9,
          6,  406,   27,    4,    1,  342,  713, 1317,   90,   16,    3,
         78,  174,  694, 4910,    2, 2556, 3599,    3,  399,  227,   31,
       4033, 2628,  441,   20,   24,  288,    7,    7,    9,    6,  144,
          5,  114,  871,  221,  922,   43,   22,   25, 3639, 1897,   27,
        217,    1, 9206,   20, 1306,    4,  258,    5,  197,   48,    6,
         20,    9,  631,  411,   11,   19,  405,   18,    8,  614,    9,
       1003,  405,   43,   22,   62,  103,   11,   19,   27,   67,  380,
         12,    9,   80,   26,   14,  152,    2, 1451,    3, 2997,  153,
         36,  146])

我们同样需要将测试集转换成代号。

In [21]:
x_test_tokens = tokenizer.texts_to_sequences(x_test_text)

## 填充和截断数据

在循环神经网络可以将任意长度的序列作为输入，但是为了整批地使用数据，序列必须有相同的长度。下面有两种形式可以实现：（A）要么我们确保整个数据的序列的长度是一样的，（B）或我们编写一个自定义数据生成器（data-generator），确保每个批次的序列具有相同的长度。。

方案（A)比较简单，但是如果我们使用数据集中最长序列的长度，我们会浪费很多内存。这问题对于较大的数据集尤其重要。

所以为了达成妥协，我们将使用一个长度来覆盖据集中的大多数序列的长度，然后我们将截断长的序列和填充短的序列。

首先我们计算数据集中所有序列的代号数量。


In [22]:
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
num_tokens = np.array(num_tokens)

一个序列中代号的平均个数是：

In [23]:
np.mean(num_tokens)

221.27716

一个序列中具有最多的代号数目是：

In [24]:
np.max(num_tokens)

2209

我们将允许最大代号数目设置为平均值加上两倍标准差。

In [25]:
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)
max_tokens

544

这涵盖了大约95%的数据集

In [26]:
np.sum(num_tokens < max_tokens) / len(num_tokens)

0.94534

当填充和截断不同长度的序列时，我们需要决定如果我们做的填充或者截断选择的是 'pre'还是 'post'。如果序列被截断，这意味着序列的一部分被简单地扔掉了。如果序列被填充，意味着0被加入到了序列中。

所以选测'pre'还是 'post'是很重要的，因为它决定了我们做截断时丢弃的是序列的头还是尾，做填充时填补的0是在序列的头部还是尾部。这可能混淆循环神经网络。

In [27]:
pad = 'pre'

In [28]:
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens,
                            padding=pad, truncating=pad)

In [29]:
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens,
                           padding=pad, truncating=pad)

现在，我们已经将训练集转换成一个大的整数矩阵（代号）：

In [30]:
x_train_pad.shape

(25000, 544)

测试集矩阵的形状：

In [31]:
x_test_pad.shape

(25000, 544)

举个例子，我们已经有了下面的代号形式的序列：

In [32]:
np.array(x_train_tokens[1])

array([   3,  591,  929,    7,    7,   48,   67,   10,  131,   11,    6,
          3,  393,   19,   12,   10,   67,  103,  121,    2,  121,    9,
          6,  406,   27,    4,    1,  342,  713, 1317,   90,   16,    3,
         78,  174,  694, 4910,    2, 2556, 3599,    3,  399,  227,   31,
       4033, 2628,  441,   20,   24,  288,    7,    7,    9,    6,  144,
          5,  114,  871,  221,  922,   43,   22,   25, 3639, 1897,   27,
        217,    1, 9206,   20, 1306,    4,  258,    5,  197,   48,    6,
         20,    9,  631,  411,   11,   19,  405,   18,    8,  614,    9,
       1003,  405,   43,   22,   62,  103,   11,   19,   27,   67,  380,
         12,    9,   80,   26,   14,  152,    2, 1451,    3, 2997,  153,
         36,  146])

它被简单地填充成下面的序列。注意当它被输入循环神经网络，它首先会输入很多0。如果我们采用'post'填充，它会先输入整形代号，然后输入0.这可能混淆循环神经网络。

In [33]:
x_train_pad[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

## 分词器逆映射

由于一些奇怪的原因，Keras的分词器没有实现将整形代号返回到单词的逆映射，当我们从代号列表重建文本字符串时需要用到它，所以我们在这创建这个映射。

In [34]:
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))

用于将代号列表转换成字符串的辅助函数。

In [35]:
def tokens_to_string(tokens):
    # Map from tokens back to words.
    words = [inverse_map[token] for token in tokens if token != 0]
    
    # Concatenate all words.
    text = " ".join(words)

    return text

举个例子，下面是数据集中的原始文本：

In [36]:
x_train_text[1]

'A simple comment...<br /><br />What can I say... this is a wonderful film that I can watch over and over. It is definitely one of the top ten comedies made. With a great cast, Jack Lemmon and Walter Matthau wording a perfect script by Neil Simon, based on his play.<br /><br />It is real to life situation done perfectly. If you have digital cable, one gets the menu on bottom of screen to give what is on. It usually gives this film ***% stars but in reality it deserves **** stars. If you really watch this film, one can tell that it will be as funny and fresh a hundred years from now.'

通过将代号列表转换到文字，我们可以重建这个文本（除了标点和其它符号）

In [37]:
tokens_to_string(x_train_tokens[1])

'a simple comment br br what can i say this is a wonderful film that i can watch over and over it is definitely one of the top ten comedies made with a great cast jack lemmon and walter matthau a perfect script by neil simon based on his play br br it is real to life situation done perfectly if you have digital cable one gets the menu on bottom of screen to give what is on it usually gives this film stars but in reality it deserves stars if you really watch this film one can tell that it will be as funny and fresh a hundred years from now'

## 创建循环神经网络

我们现在准备创建循环神经网络（RNN），我们将用Keras的API，因为它很简单。Keras的教程详见教程#03-C。

In [38]:
model = Sequential()

RNN的第一层称为embedding层，它将整形代号转换成一个向量的值。这是必须的，因为整形代号可以取0到10000的值来表示1万字的词汇表。RNN无法接受这么大范围的输入。embedding层被当做RNN的一部分来寻来，它将学习如何将具有类似语义含义的词映射到相似的嵌入向量（embedding-vectors），下面会进一步展示。

首先我们为整形代号定义嵌入向量（embedding-vectors）的尺寸。在这个例子中，我们设置为8，所以每个整形代号将会被转成长度为8的向量。嵌入向量（embedding-vectors）的值会逐渐的接近-1.0到1.0，尽管它们可能在某种程度上超过了这个范围。

嵌入向量（embedding-vectors）的尺寸通常在100-300之间选择，但它似乎在情绪分析上运行得相当不错。

In [39]:
embedding_size = 8

embedding层也需要知道词汇表中词的数量(`num_words`)，和填充的代号序列的长度(`max_tokens`)。我们要给这一层一个名字，因为我们需要获取它的权重。

In [40]:
model.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                    name='layer_embedding'))

我们现在可以加入第一个GRU到网络中，它会有16个输出单元。因为我们之后会加入第二个GUR，需要返回上数据的序列，作为下一个GRU序列的输入。

In [41]:
model.add(GRU(units=16, return_sequences=True))

Instructions for updating:
keep_dims is deprecated, use keepdims instead


这里加入第二个GRU，有8个输出单元。因为接下去还有另一层GRU，所以也必须返回序列。

In [42]:
model.add(GRU(units=8, return_sequences=True))

我们加入第三个，也是最后一个GUR，它有4个输出单元。它之后会接一个全连接层，所以它应该给出GRU最后的输出，而不是一个序列。

In [43]:
model.add(GRU(units=4))

加入一个全连接层，计算一个0.0到1.0的值，用于分类。

In [44]:
model.add(Dense(1, activation='sigmoid'))

用一个给定学习率的Adam优化器。

In [45]:
optimizer = Adam(lr=1e-3)

编译Keras模型，准备好训练。

In [46]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [47]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer_embedding (Embedding)  (None, 544, 8)            80000     
_________________________________________________________________
gru_1 (GRU)                  (None, None, 16)          1200      
_________________________________________________________________
gru_2 (GRU)                  (None, None, 8)           600       
_________________________________________________________________
gru_3 (GRU)                  (None, 4)                 156       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 5         
Total params: 81,961
Trainable params: 81,961
Non-trainable params: 0
_________________________________________________________________


## 训练循环神经网络

我们现在可以训练这个模型。注意，我们用的数据集是被填充过的序列。我们用了训练集中5%作为验证集，所以我们会有一个大致的判断，这个模型是具有通用性的还是对训练集过拟合的。

In [48]:
%%time
model.fit(x_train_pad, y_train,
          validation_split=0.05, epochs=3, batch_size=64)

Train on 23750 samples, validate on 1250 samples
Epoch 1/3
23750/23750 [==============================]23750/23750 [==============================] - 464s 20ms/step - loss: 0.6517 - acc: 0.6002 - val_loss: 0.6218 - val_acc: 0.6752

Epoch 2/3
23750/23750 [==============================]23750/23750 [==============================] - 447s 19ms/step - loss: 0.4292 - acc: 0.8102 - val_loss: 0.6701 - val_acc: 0.6512

Epoch 3/3
23750/23750 [==============================]23750/23750 [==============================] - 445s 19ms/step - loss: 0.3092 - acc: 0.8765 - val_loss: 0.3182 - val_acc: 0.8752

CPU times: user 35min 19s, sys: 2min 41s, total: 38min
Wall time: 22min 37s


## 测试集上的性能

现在，模型已经被训练好了，我们可以计算测试集上分类的正确率。

In [49]:
%%time
result = model.evaluate(x_test_pad, y_test)

25000/25000 [==============================]25000/25000 [==============================] - 175s 7ms/step

CPU times: user 2min 59s, sys: 340 ms, total: 2min 59s
Wall time: 2min 55s


In [50]:
print("Accuracy: {0:.2%}".format(result[1]))

Accuracy: 86.71%


## 错误分类的文本案例

为了展示错误分类的文本案例，我们首先计算了测试集中的前1000个文本的预测情绪。


In [51]:
%%time
y_pred = model.predict(x=x_test_pad[0:1000])
y_pred = y_pred.T[0]

CPU times: user 7.01 s, sys: 0 ns, total: 7.01 s
Wall time: 6.88 s


这些预测的数字介于0.0到1.0。我们用一个阈值判断，当值大于0.5则被视为1.0，所有小于0.5的值被认为是0.0。我们的预测变成只有0.0或1.0。

In [52]:
cls_pred = np.array([1.0 if p>0.5 else 0.0 for p in y_pred])

测试集中前1000个文本正确的“类”需要拿来做比较。

In [53]:
cls_true = np.array(y_test[0:1000])

然后我们将这两个数组比较，得出不正确的分类的索引。

In [54]:
incorrect = np.where(cls_pred != cls_true)
incorrect = incorrect[0]

对这1000个文本，有多少个被错误分类？

In [55]:
len(incorrect)

121

让我们看看第一个被错误分类的文本。我们需要用到它的索引几次。

In [56]:
idx = incorrect[0]
idx

13

被错误分类的文本是：

In [57]:
text = x_test_text[idx]
text

'I would like to start by saying I can only hope that the makers of this movie and it\'s sister film The Intruder (directed by the great unheralded stylist auteur that is Jopi Burnama) know in their hearts just how much pleasure they have brought to me and my friends in the sleepy north eastern town of Jarrow.<br /><br />From the opening pre credit sequence which manages to drag ever so slightly despite containing a man crashing through a window on a motorbike, the pitiless destruction of a silence lab, the introduction of one of the most simultaneously annoying and anaemic bad guys in movie history and costume design that Jean Paul Gautier would find ott and garish. Make no mistake; this is a truly unique experience. Early highlight - an explosion (get used to it, plenty more where that came from!) followed by a close up of our chubby heroine and the most hilarious line reading of the word "dad" in living memory. And then... the theme song...<br /><br />Yeah, this deserves its own par

下面预测和正确的类别：

In [58]:
y_pred[idx]

0.08332923

In [59]:
cls_true[idx]

1.0

## 新的数据

让我们试着对我们编造的新文本进行分类。它们有些很明显，而另一些则使用否定和讽刺方式来试图混淆模型，将文本错误分类。

In [60]:
text1 = "This movie is fantastic! I really like it because it is so good!"
text2 = "Good movie!"
text3 = "Maybe I like this movie."
text4 = "Meh ..."
text5 = "If I were a drunk teenager then this movie might be good."
text6 = "Bad movie!"
text7 = "Not a good movie!"
text8 = "This movie really sucks! Can I get my money back please?"
texts = [text1, text2, text3, text4, text5, text6, text7, text8]

首先，我们将文本转换成整形代号的数组。

In [61]:
tokens = tokenizer.texts_to_sequences(texts)

为了将不同长度的文本输入到模型，我们也需要填充或截断它们。

In [62]:
tokens_pad = pad_sequences(tokens, maxlen=max_tokens,
                           padding=pad, truncating=pad)
tokens_pad.shape

(8, 544)

我们现在可以使用训练好的模型去预测这些文本的情感。

In [63]:
model.predict(tokens_pad)

array([[0.868934  ],
       [0.72526425],
       [0.33099633],
       [0.49190348],
       [0.3054021 ],
       [0.14959489],
       [0.5235635 ],
       [0.21565402]], dtype=float32)

结果仅仅0.0意味着是负面的情感，结果接近1.0意味着是正面的情感。每次你训练模型时，这些数字都会有所不同。

## Embeddings

这个模型不能直接输入整形代号的序列，因为它们是整形的数值，范围从0到词汇表中词的个数，例如10000。所以我们需要将整形的代号转换成一个值处于-1.0到1.0之间的向量，它可以作为神经网络的输入。

这个从整形代号转换到实数向量的映射被称为 "embedding"。它的本质是一个矩阵，每一行都包含单个代号的矢量映射。这意味着我们可以通过用代号作为矩阵中的索引，快速查找每个整形代号的映射。在训练过程中，embeddings是和模型的其它部分一起学习的。

理想的embedding会学习到一个映射，让在意义上相似的词也有相似的嵌入值（embedding-vectors）。让我们看一下是否发生了这种情况。

首先我们需要从模型中获得embedding层：

In [64]:
layer_embedding = model.get_layer('layer_embedding')

我们可以通过embedding层获得用于完成映射的权重。

In [65]:
weights_embedding = layer_embedding.get_weights()[0]

注意，这些权重实际上只是一个矩阵（词汇表的词数，嵌入矢量长度）。它本质就是一个查找矩阵。

In [66]:
weights_embedding.shape

(10000, 8)

让我们获取 'good'的整形代号，就是它在词汇表中的索引。

In [67]:
token_good = tokenizer.word_index['good']
token_good

49

让我们获取 'great'的整形代号。

In [68]:
token_great = tokenizer.word_index['great']
token_great

78

由于依据在数据集中出现的频率，这些整形代号的值可能相差的比较远。

现在，让我们比较 'good'和'great'的嵌入向量（embedding-vectors）。其中一些值是相似的，也有些值是完全不同的。注意，每次您对模型进行训练时，这些值都会发生变化。


In [69]:
weights_embedding[token_good]

array([0.86528164, 0.6867993 , 0.4362397 , 0.66128314, 0.11546915,
       0.94507647, 0.32628497, 0.535881  ], dtype=float32)

In [70]:
weights_embedding[token_great]

array([ 1.0691622 ,  1.124244  , -0.04477464, -0.05861434,  0.16965319,
        1.2626944 ,  0.76136374, -0.00998422], dtype=float32)

同样的，我们也可以比较'bad' 和 'horrible'。

In [71]:
token_bad = tokenizer.word_index['bad']
token_horrible = tokenizer.word_index['horrible']

In [72]:
weights_embedding[token_bad]

array([ 0.31903917,  0.53934103,  1.3727672 ,  1.4083829 ,  0.8475107 ,
       -0.22946651,  0.0251075 ,  0.77032244], dtype=float32)

In [73]:
weights_embedding[token_horrible]

array([ 0.47915924,  0.12226178,  0.90192014,  0.742338  ,  0.58730644,
        0.32736972, -0.17633988,  1.3744307 ], dtype=float32)

### 词排序

我们也可以将词汇表中的词按“相似性”进行排列。我们想要看看是否具有相似嵌入向量（embedding-vectors）的词也有相似的含义。

嵌入向量embedding-vectors的相似性可以通过不同方法度量，比如，欧式距离或者余弦距离。

我们设计一个辅助函数用于计算这些距离，然后按序打印出这些词。

In [74]:
def print_sorted_words(word, metric='cosine'):
    """
    Print the words in the vocabulary sorted according to their
    embedding-distance to the given word.
    Different metrics can be used, e.g. 'cosine' or 'euclidean'.
    """

    # Get the token (i.e. integer ID) for the given word.
    token = tokenizer.word_index[word]

    # Get the embedding for the given word. Note that the
    # embedding-weight-matrix is indexed by the word-tokens
    # which are integer IDs.
    embedding = weights_embedding[token]

    # Calculate the distance between the embeddings for
    # this word and all other words in the vocabulary.
    distances = cdist(weights_embedding, [embedding],
                      metric=metric).T[0]
    
    # Get an index sorted according to the embedding-distances.
    # These are the tokens (integer IDs) for words in the vocabulary.
    sorted_index = np.argsort(distances)
    
    # Sort the embedding-distances.
    sorted_distances = distances[sorted_index]
    
    # Sort all the words in the vocabulary according to their
    # embedding-distance. This is a bit excessive because we
    # will only print the top and bottom words.
    sorted_words = [inverse_map[token] for token in sorted_index
                    if token != 0]

    # Helper-function for printing words and embedding-distances.
    def _print_words(words, distances):
        for word, distance in zip(words, distances):
            print("{0:.3f} - {1}".format(distance, word))

    # Number of words to print from the top and bottom of the list.
    k = 10

    print("Distance from '{0}':".format(word))

    # Print the words with smallest embedding-distance.
    _print_words(sorted_words[0:k], sorted_distances[0:k])

    print("...")

    # Print the words with highest embedding-distance.
    _print_words(sorted_words[-k:], sorted_distances[-k:])

然后我们可以打印出距离'great'从近到远的词。请注意，每次您对模型进行训练时，这些都可能发生变化。

In [75]:
print_sorted_words('great', metric='cosine')

Distance from 'great':
0.000 - great
0.016 - touching
0.017 - arguments
0.025 - nevertheless
0.031 - elmer
0.032 - 8
0.036 - ritter
0.037 - juliet
0.041 - randy
0.045 - afterward
...
1.057 - rubbish
1.060 - dull
1.064 - disappointing
1.069 - unlikeable
1.078 - uninspired
1.083 - lacks
1.188 - worst
1.225 - waste
1.247 - awful
1.282 - terrible


相似的，我们也可以打印出距离 'worst'从近到远的词。

In [76]:
print_sorted_words('worst', metric='cosine')

Distance from 'worst':
0.000 - worst
0.047 - embarrassingly
0.053 - terrible
0.094 - retarded
0.095 - poor
0.095 - stereotyping
0.096 - uninspired
0.099 - awful
0.100 - severed
0.108 - lacks
...
1.167 - restraint
1.168 - available
1.176 - foremost
1.188 - great
1.193 - mesmerizing
1.222 - highly
1.229 - exploration
1.239 - delightful
1.268 - wonderfully
1.323 - 7


## 总结

这份教程展示了使用了带有整形代号（integer-tokens）和embedding层的循环神经网络进行自然语言处理(NLP)的基本方法。用于IMDB影评的情感分析。如果超参数选择合适，它工作的相当好。但是重要的是明白它并不是像人一样去理解文本。这个系统并不能真正的理解文本。这只是一种巧妙的模式识别方法。

## 练习

下面使一些可能会让你提升TensorFlow技能的一些建议练习。为了学习如何更合适地使用TensorFlow，实践经验是很重要的。

在你对这个Notebook进行修改之前，可能需要先备份一下。

* 训练的更久。性能是否提高了？
* 如果你的模型对训练集过拟合，尝试使用dropout层并在GRU中也设置dropout。
* 增加或减少词汇表中词汇的数量。这是当`Tokenizer`初始化时完成的。是否影响性能？
* 增加嵌入向量（ embedding-vectors ）的尺寸，例如 200.是否影响性能？
* 尝试给循环神经网络不同的超参数。
* 利用教程 #19 中的贝叶斯最优化去找到最好超参数。
* 在 `pad_sequences()`中选择'post'完成填充和截断。是否影响性能？
* 使用单独的字符代替记号化单词作为词汇表。然后你可以为每个字符使用一个热编码的向量，代替使用embedding层。
* 使用`model.fit_generator()` 代替`model.fit()`并制作你自己的数据生成器（data-generator），它可以用 `x_train_tokens`的随机子集作为一个批，这些序列必须被填充，所以它们都与最长的序列相匹配。
* 向朋友解释程序如何工作。

## License (MIT)

Copyright (c) 2018 by [Magnus Erik Hvass Pedersen](http://www.hvass-labs.org/)

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.